In [1]:
import sqlite3
import json

In [2]:

with sqlite3.connect("dados.db") as con:
    cur = con.cursor()
    cur.execute("""CREATE TABLE IF NOT EXISTS dados_busca 
                (id INTEGER PRIMARY KEY AUTOINCREMENT, data_raw TEXT)
    """)


In [3]:
with open("dados.json", "r") as f:
    data = json.dumps(json.load(f), ensure_ascii=False)
    with sqlite3.connect("dados.db") as con:
        cur = con.cursor()
        cur.execute("""INSERT INTO dados_busca (data_raw) 
                    VALUES (?)""", (data,)
        )
        
        con.commit()


# sem isso cria dados.db-journal que é o registro de transações garantido que seja feito de forma segura e atomica


In [4]:

with sqlite3.connect("dados.db") as con:
    cur = con.cursor()
    cur.execute("""SELECT * 
                FROM dados_busca
    """)

    row = cur.fetchone()
    
    print(row)



(1, '{"event_time": "2025-06-03T10:15:00Z", "user_id": 12345, "session_id": "abcde-12345", "event_type": "product_view", "product": {"id": 9876, "name": "Smartphone XYZ", "category": "Electronics", "price": 1999.99}, "device": {"type": "mobile", "os": "Android"}, "location": {"country": "Brazil", "city": "São Paulo"}}')


In [5]:
with sqlite3.connect("dados.db") as con:
    cur = con.cursor()
    cur.execute("SELECT json_extract(data_raw, '$.session_id') AS ID_sessao," \
                " json_extract(data_raw, '$.event_type') AS tipo_evento," \
                " json_extract(data_raw, '$.product.name') AS nome_produto," \
                " json_extract(data_raw, '$.product.category') AS categoria," \
                " json_extract(data_raw, '$.location.country') AS pais " \
                " FROM dados_busca"
        )
    
    row = cur.fetchone()

    print(row)


('abcde-12345', 'product_view', 'Smartphone XYZ', 'Electronics', 'Brazil')


In [6]:
data_product_view = {
  "user_id": 12345,
  "session_id": "abcde-12345",
  "event_type": "cart_view",
  "products": [
    {"id": 9876, "name": "Smartphone XYZ", "category": "Electronics"},
    {"id": 1234, "name": "Notebook ABC", "category": "Computers"}
  ]
}

with sqlite3.connect("dados.db") as con:
    cur = con.cursor()
    cur.execute("""INSERT INTO dados_busca (data_raw) 
                VALUES (?)""", [json.dumps(data_product_view, ensure_ascii=False)]
    )

In [7]:
with sqlite3.connect("dados.db") as con:
    cur = con.cursor()
    cur.execute("""SELECT data_raw 
                FROM dados_busca 
                WHERE id == 2
    """)

    row = cur.fetchall()

    print(row)

[('{"user_id": 12345, "session_id": "abcde-12345", "event_type": "cart_view", "products": [{"id": 9876, "name": "Smartphone XYZ", "category": "Electronics"}, {"id": 1234, "name": "Notebook ABC", "category": "Computers"}]}',)]


In [8]:
with sqlite3.connect("dados.db") as con:
    cur = con.cursor()

    cur.execute("""SELECT json_extract(data_raw, '$.session_id'),
                json_extract(product.value, '$.id') AS product_id,
                json_extract(product.value, '$.name') AS product_name,
                json_extract(product.value, '$.category') AS category
                FROM dados_busca,
                json_each(json_extract(data_raw, '$.products')) product
    """)

    row = cur.fetchall()

    print(row)



[('abcde-12345', 9876, 'Smartphone XYZ', 'Electronics'), ('abcde-12345', 1234, 'Notebook ABC', 'Computers')]


In [9]:
with sqlite3.connect("dados.db") as con:
    cur = con.cursor()

    cur.execute("""CREATE TABLE IF NOT EXISTS produtos_eventos AS
                SELECT (SELECT json_extract(data_raw, '$.event_time')
                         FROM dados_busca
                        ) AS event_time,
                json_extract(data_raw,'$.user_id') AS user_id,
                json_extract(data_raw,'$.session_id') AS session_id,
                json_extract(data_raw,'$.event_type') AS event_type,
                CASE
                    WHEN json_extract(data_raw,'$.event_type') = 'product_view'
                        THEN json_extract(data_raw,'$.product.id')
                        ELSE json_extract(product.value,'$.id')
                END AS product_id,
                CASE
                    WHEN json_extract(data_raw,'$.event_type') = 'product_view'
                        THEN json_extract(data_raw,'$.product.name')
                        ELSE json_extract(product.value,'$.name')
                END AS product_name,
                CASE
                    WHEN json_extract(data_raw,'$.event_type') = 'product_view'
                        THEN json_extract(data_raw,'$.product.category')
                        ELSE json_extract(product.value,'$.category')
                END AS product_category,
                CASE
                    WHEN json_extract(data_raw,'$.event_type') = 'product_view'
                        THEN json_extract(data_raw,'$.product.price')
                        ELSE json_extract(product.value,'$.price')
                END AS product_price,
                CASE
                    WHEN json_extract(data_raw,'$.event_type') = 'product_view'
                        THEN json_extract(data_raw,'$.device.type')
                        ELSE json_extract(product.value,'$.type')
                END AS product_type,
                (SELECT json_extract(data_raw, '$.location.country')
                         FROM dados_busca
                        ) AS country,
                (SELECT json_extract(data_raw, '$.location.city')
                         FROM dados_busca
                        ) AS city
                FROM dados_busca
                LEFT JOIN
                json_each(json_extract(data_raw, '$.products')) product
    """)
    
    cur.execute("""SELECT 
                *
                FROM
                produtos_eventos
    """)
    
    row = cur.fetchall()

    print(row)

    

[('2025-06-03T10:15:00Z', 12345, 'abcde-12345', 'product_view', 9876, 'Smartphone XYZ', 'Electronics', 1999.99, 'mobile', 'Brazil', 'São Paulo'), ('2025-06-03T10:15:00Z', 12345, 'abcde-12345', 'cart_view', 9876, 'Smartphone XYZ', 'Electronics', None, None, 'Brazil', 'São Paulo'), ('2025-06-03T10:15:00Z', 12345, 'abcde-12345', 'cart_view', 1234, 'Notebook ABC', 'Computers', None, None, 'Brazil', 'São Paulo')]


In [13]:
with sqlite3.connect("dados.db") as con:
    cur = con.cursor()

    cur.execute("""SELECT
        json_object(
          'user_id', user_id,
          'session_id', session_id,
          'product', 
          json_object(
            'id', product_id,
            'name', product_name
          ),
          'event_type', event_type                
        ) AS json_object
                
        FROM produtos_eventos
    """)

    row = cur.fetchall()

    print(row)

[('{"user_id":12345,"session_id":"abcde-12345","product":{"id":9876,"name":"Smartphone XYZ"},"event_type":"product_view"}',), ('{"user_id":12345,"session_id":"abcde-12345","product":{"id":9876,"name":"Smartphone XYZ"},"event_type":"cart_view"}',), ('{"user_id":12345,"session_id":"abcde-12345","product":{"id":1234,"name":"Notebook ABC"},"event_type":"cart_view"}',)]
